## Heroes Of Pymoli Data Analysis
* Of the 573 active players, the vast majority are male (81%). There also exists, a smaller, but notable proportion of female players (17%).

* Our peak age demographic falls between 20-24 (45%) with secondary groups falling between 15-19 (17%) and 25-29 (15%).

* Our players are putting in significant cash during the lifetime of their gameplay.   
-----

In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# File to Load (Remember to Change These)
path = './raw_data/'
json1 = 'purchase_data.json'
json2 = 'purchase_data2.json'

In [3]:
df1 = pd.read_json(path+json1)
df2 = pd.read_json(path+json2)

In [4]:
df = pd.merge(df1, df2, how='left')

# Player Count

In [5]:
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [6]:
df.columns

Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object')

In [7]:
count_players = df.SN.nunique()
count_players

573

In [8]:
pd.DataFrame({ 'Total Players': [count_players] })

,Total Players
0,573


# Purchasing Analysis (Total)

In [9]:
cols = ['Number of Unique Items', 'Average Price', 'Number of Purchases', 
       'Total Revenue']
cols

['Number of Unique Items',
 'Average Price',
 'Number of Purchases',
 'Total Revenue']

In [10]:
analysis = [[df1['Item ID'].nunique(), round(df1.Price.mean(),2), df1.Price.count(), df1.Price.sum()]]
analysis

[[183, 2.93, 780, 2286.33]]

In [11]:
newDF = pd.DataFrame(analysis, columns=cols)
newDF

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,2.93,780,2286.33


# Gender Demographics

In [12]:
# drop duplicates

drop_duplicates = df1[['SN','Gender', 'Age']].drop_duplicates()

In [13]:
player_countByGender = drop_duplicates['Gender'].value_counts()
player_countByGender

Male                     465
Female                   100
Other / Non-Disclosed      8
Name: Gender, dtype: int64

In [14]:
byGenderPercentage = player_countByGender / count_players * 100
byGenderPercentage

Male                     81.151832
Female                   17.452007
Other / Non-Disclosed     1.396161
Name: Gender, dtype: float64

In [15]:
genderDemog = (pd.DataFrame({'Percentage of Players': byGenderPercentage,
                           'Total Count': player_countByGender})).round(2)
genderDemog

,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


# Purchasing Analysis (Gender)

In [16]:
purchaseByGender = df1.groupby('Gender').sum()['Price']
purchaseByGender

Gender
Female                    382.91
Male                     1867.68
Other / Non-Disclosed      35.74
Name: Price, dtype: float64

In [17]:
genderPurchaseAverage = round(df1.groupby('Gender').mean()['Price'],2)
genderPurchaseAverage

Gender
Female                   2.82
Male                     2.95
Other / Non-Disclosed    3.25
Name: Price, dtype: float64

In [18]:
genderPurchaseCount = df1.groupby('Gender').count()['Price']
genderPurchaseCount

Gender
Female                   136
Male                     633
Other / Non-Disclosed     11
Name: Price, dtype: int64

In [19]:
# calculate by dividing total purchase value(sum) by the #total number of players by genger

pCount_normalize = round(purchaseByGender / player_countByGender,2)
pCount_normalize

Female                   3.83
Male                     4.02
Other / Non-Disclosed    4.47
dtype: float64

In [20]:
values = ({
    'Average Purchase Price':genderPurchaseAverage, 
    'Normalized Total':pCount_normalize, 
    'Purchase Count':genderPurchaseCount, 
    'Total Purchase Value':purchaseByGender
})
    
PurchaseDF= pd.DataFrame(values)
PurchaseDF

,Average Purchase Price,Normalized Total,Purchase Count,Total Purchase Value
Gender,,,,
Female,2.82,3.83,136,382.91
Male,2.95,4.02,633,1867.68
Other / Non-Disclosed,3.25,4.47,11,35.74


In [21]:
#Organize Table

organized_purchaseDF = PurchaseDF[['Purchase Count', 'Average Purchase Price','Total Purchase Value', 'Normalized Total']]
organized_purchaseDF

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Total
Gender,,,,
Female,136,2.82,382.91,3.83
Male,633,2.95,1867.68,4.02
Other / Non-Disclosed,11,3.25,35.74,4.47


# Age Demographics

In [22]:
#Add Column on the players dataframe created earlier on the 1st step:

age_bracket = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '>40']
bins = [0, 9.5, 14.5, 19.5, 24.5, 29.5, 34.5, 39.5, 110]

drop_duplicates['Age Bracket'] = pd.cut(drop_duplicates['Age'], bins, labels=age_bracket)
drop_duplicates.head()

,SN,Gender,Age,Age Bracket
0,Aelalis34,Male,38,35-39
1,Eolo46,Male,21,20-24
2,Assastnya25,Male,34,30-34
3,Pheusrical25,Male,21,20-24
4,Aela59,Male,23,20-24


In [23]:
age_bracketTotals = drop_duplicates['Age Bracket'].value_counts()
age_bracketTotals

20-24    259
15-19    100
25-29     87
30-34     47
35-39     27
10-14     23
<10       19
>40       11
Name: Age Bracket, dtype: int64

In [24]:
percentage_ageBracket = age_bracketTotals / count_players *100
percentage_ageBracket

20-24    45.200698
15-19    17.452007
25-29    15.183246
30-34     8.202443
35-39     4.712042
10-14     4.013962
<10       3.315881
>40       1.919721
Name: Age Bracket, dtype: float64

In [25]:
ageDemographics = pd.DataFrame({
    'Percentage of Players': percentage_ageBracket,
    'Total Count': age_bracketTotals
})

ageDemographics.sort_index()

,Percentage of Players,Total Count
<10,3.315881,19
10-14,4.013962,23
15-19,17.452007,100
20-24,45.200698,259
25-29,15.183246,87
30-34,8.202443,47
35-39,4.712042,27
>40,1.919721,11


# Purchasing Analysis (Age)

In [26]:
age_bracket = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '>40']
bins = [0, 9.5, 14.5, 19.5, 24.5, 29.5, 34.5, 39.5, 110]

df1['Age Bracket'] = pd.cut(df1['Age'], bins, labels=age_bracket)
df1.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Bracket
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [27]:
player_countByAge = df1['Age Bracket'].value_counts()

In [28]:
purchaseByAge = df1.groupby('Age Bracket').sum()['Price']
agePurchaseAverage = round(df1.groupby('Age Bracket').mean()['Price'],2)
agePurchaseCount = df1.groupby('Age Bracket').count()['Price']
ACount_normalize = round(purchaseByAge / player_countByAge,2)

In [29]:
purchasing_analysis = pd.DataFrame({
    'Purchase Count':agePurchaseCount,
    'Average Purchase Price':agePurchaseAverage,
    'Total Purchase Value':purchaseByAge,
    'Normalized Totals':ACount_normalize
})

purchasing_analysis

,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Value
10-14,2.77,2.77,35,96.95
15-19,2.91,2.91,133,386.42
20-24,2.91,2.91,336,978.77
25-29,2.96,2.96,125,370.33
30-34,3.08,3.08,64,197.25
35-39,2.84,2.84,42,119.40
<10,2.98,2.98,28,83.46
>40,3.16,3.16,17,53.75


# Top Spenders

In [30]:
df1.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Bracket
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [31]:

purchaseTotal = df1.groupby(['SN']).sum()['Price']
purchaseAverage = df1.groupby(['SN']).mean()['Price']
purchaseCount = df1.groupby(['SN']).count()['Price']

user_data = pd.DataFrame({'Total Purchase Value': purchaseTotal,
                          'Average Purchase Price': purchaseAverage,
                          'Purchase Count': purchaseCount})


user_data.sort_values('Total Purchase Value', ascending=False).head(5)

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Undirrala66,3.412000,5,17.06
Saedue76,3.390000,4,13.56
Mindimnya67,3.185000,4,12.74
Haellysu29,4.243333,3,12.73
Eoda93,3.860000,3,11.58


# Most Popular Items

In [32]:
pValue = df1.groupby(['Item ID', 'Item Name']).sum()['Price']
pPrice = df1.groupby(['Item ID', 'Item Name']).mean()['Price']
pCount = df1.groupby(['Item ID', 'Item Name']).count()['Price']

popular = pd.DataFrame({'Total Purchase Value': pValue,
                          'Item Price': pPrice,
                          'Purchase Count': pCount})


popular.sort_values('Purchase Count', ascending=False).head(5)

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",2.35,11,25.85
84,Arcane Gem,2.23,11,24.53
31,Trickster,2.07,9,18.63
175,Woeful Adamantite Claymore,1.24,9,11.16
13,Serenity,1.49,9,13.41


# Most Profitable Items

In [33]:
popular.sort_values('Total Purchase Value', ascending=False).head(5)

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,4.14,9,37.26
115,Spectral Diamond Doomblade,4.25,7,29.75
32,Orenmir,4.95,6,29.70
103,Singed Scalpel,4.87,6,29.22
107,"Splitter, Foe Of Subtlety",3.61,8,28.88
